In [1]:
from bs4 import BeautifulSoup
import random, re, requests, json, time
from datetime import datetime
import pandas as pd

In [2]:
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.1 Safari/605.1.15'}
headers1 = {"User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36"}
headers2 = {"User-Agent":"Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:73.0) Gecko/20100101 Firefox/73.0"}

In [3]:
def cleanlink(url):
    url = url.split('?')[0]
    url = url.split('/')[:5]
    return('/'.join(url)+'/')

In [4]:
def writeBookinfo(url):
    bookhtml = requests.get(booklink, headers = headers2)
    bookinfo = BeautifulSoup(bookhtml.content, features="html5lib")
    
    try:
        result = bookinfo.find('script', {'type':'application/ld+json'})
        result = json.loads(result.text.strip())
    except:
        return(False)
    try:
        bookName = result['name']
    except:
        bookName = ''
    try:	    
        bookAuthor = result['author'][0]['name']
    except:
        bookAuthor = ''
    try:
        bookIsbn = result['isbn']
    except:
        bookIsbn = ''
    try:
        bookUrl = result['url']
        if bookUrl != url:
            print(bookUrl + '<--->' + url)
    except:
        bookUrl = url
    

    
    try:
        voteNum = bookinfo.find('span',{'property':'v:votes'})
        voteNum = voteNum.text.strip()
    except:
        voteNum = '0'
        
    try:
        voteAvg = bookinfo.find('strong',{'property':'v:average'})
        voteAvg = voteAvg.text.strip()
    except:
        voteAvg = ''
        
    with open('BookList.csv', mode = 'a') as f:
        f.write(bookIsbn+',')
        f.write(bookName+',')
        f.write(bookAuthor+',')
        f.write(voteAvg+',')
        f.write(voteNum+',')
        f.write(bookUrl+',')
        f.write(str(datetime.date(datetime.today()))+'\n')
        f.close()
        print('Book Info is written into file! Take a short break!')
    time.sleep(random.randint(1,3))
    
    return(True)

In [5]:
def updateLinkset(booklinkset):
    try:
        b = open('booklinkset.txt','r')
        booklinkset = b.readlines()
        b.close()
        booklinkset = set([i.split('?')[0] for i in booklinkset])
        print('Book Link set is updated.')
        return(booklinkset)
    except:
        return(set()) 

In [6]:
booklinkset = updateLinkset(set())
url = 'https://book.douban.com/'
loop = 0

Book Link set is updated.


In [8]:
while True:
    try:
        print('Try to get html.')
        html = requests.get(url, headers = headers1)
        print(str(html)+'  '+str(loop))
        page = BeautifulSoup(html.content, features="html5lib")
        urllist = page.findAll('a',{'href':re.compile("https://book.douban.com/subject/[0-9]*")})
        for link in urllist:
            booklink = cleanlink(link['href'])
            if booklink not in booklinkset:
                result = writeBookinfo(booklink)
                if result: 
                    booklinkset.add(booklink)
                    a = open('booklinkset.txt','a')
                    a.write(booklink+'?\n')
                    a.close()
        url = random.sample(booklinkset,1)[0]
    except:
        print('Something Wrong')
        url = random.sample(booklinkset,1)[0]
        time.sleep(random.randint(1,2))
    loop += 1

Try to get html.
Something Wrong


KeyboardInterrupt: 

In [7]:
url = 'https://www.douban.com/'
html = requests.get(url, headers = headers)
html

<Response [200]>